In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
credicard_df = pd.read_csv('/kaggle/input/ccdata/CC GENERAL.csv')

In [3]:
credicard_df.shape

In [4]:
credicard_df.head()

In [5]:
credicard_df.info()

In [6]:
credicard_df.describe()

In [7]:
credicard_df[credicard_df['ONEOFF_PURCHASES'] == 40761.250000]

In [8]:
credicard_df[credicard_df['CASH_ADVANCE'] == 47137.211760]

In [9]:
sns.heatmap(credicard_df.isnull())

In [10]:
credicard_df.isnull().sum()

In [11]:
credicard_df.loc[(credicard_df['MINIMUM_PAYMENTS'].isnull() == True), 'MINIMUM_PAYMENTS'] = credicard_df['MINIMUM_PAYMENTS'].mean()

In [12]:
credicard_df.loc[(credicard_df['CREDIT_LIMIT'].isnull() == True), 'CREDIT_LIMIT'] = credicard_df['CREDIT_LIMIT'].mean()

In [13]:
credicard_df.isnull().sum()

In [14]:
credicard_df.drop('CUST_ID', axis=1, inplace=True)

In [15]:
credicard_df.head()

In [16]:
credicard_df.columns

In [17]:
plt.figure(figsize=(10,50))
for i in range(len(credicard_df.columns)):
    plt.subplot(17, 1, i + 1)
    sns.distplot(credicard_df[credicard_df.columns[i]], kde=True)
    plt.title(credicard_df.columns[i])
plt.tight_layout()

In [18]:
correlations = credicard_df.corr()


In [19]:
f, ax = plt.subplots(figsize=(14,15))
sns.heatmap(correlations, annot=True)

In [20]:
min(credicard_df['BALANCE']), max(credicard_df['BALANCE'])

In [21]:
scaler = StandardScaler()
credicard_df_scaled = scaler.fit_transform(credicard_df)

In [22]:
type(credicard_df_scaled) , type(credicard_df)

In [23]:
min(credicard_df_scaled[0]), max(credicard_df_scaled[0])

In [24]:
credicard_df_scaled

In [25]:
wcss_1 = []
range_values = range(1, 20)
for i in range_values:
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(credicard_df_scaled)
  wcss_1.append(kmeans.inertia_)

In [26]:
print(wcss_1)

In [27]:
plt.plot(wcss_1, 'bx-')
plt.xlabel('Clusters')
plt.ylabel('WCSS');

In [28]:
kmeans = KMeans(n_clusters=8)
kmeans.fit(credicard_df_scaled)
labels = kmeans.labels_

In [29]:
labels, len(labels)

In [30]:
np.unique(labels, return_counts=True)

In [31]:
kmeans.cluster_centers_

In [32]:
cluster_centers = pd.DataFrame(data = kmeans.cluster_centers_, columns = [credicard_df.columns])
cluster_centers

- Grupo 0 (VIP/Prime): limite do cartão alto (15570) e o mais alto percentual de pagamento da fatura completa (0.47). Aumentar o limite do cartão e o hábito de compras

- Grupo 3: Clientes que pagam poucos juros para o banco e são cuidadosos com seu dinheiro. Possui menos dinheiro na conta corrente (104) e não sacam muito dinheiro do limite do cartão (302). 23% de pagamento da fatura completa do cartão de crédito

- Grupo 5: usam o cartão de crédito como "empréstimo" (setor mais lucrativo para o banco), possuem muito dinheiro na conta corrente (5119) e sacam muito dinheiro do cartão de crédito (5246), compram pouco (0.3) e usam bastante o limite do cartão para saques (0.51). Pagam muito pouco a fatura completa (0.03)

- Grupo 7 (clientes novos): clientes mais novos (7.23) e que mantém pouco dinheiro na conta corrente (863)

In [33]:
cluster_centers = scaler.inverse_transform(cluster_centers)
cluster_centers = pd.DataFrame(data = cluster_centers, columns = [credicard_df.columns])
cluster_centers

In [34]:
labels, len(labels)

In [35]:
credicard_df_cluster = pd.concat([credicard_df, pd.DataFrame({'Cluster':labels})], axis=1)
credicard_df_cluster.head()

In [36]:
for i in credicard_df.columns:
    plt.figure(figsize=(35,5))
    for j in range(8):
        plt.subplot(1,8, j + 1)
        cluster = credicard_df_cluster[credicard_df_cluster['Cluster'] == j]
        cluster[i].hist(bins = 20)
        plt.title('{} \nCluster {}'.format(i, j))
    plt.show()

In [37]:
credit_ordered = credicard_df_cluster.sort_values(by='Cluster')
credit_ordered.head()

In [38]:
credit_ordered.tail()

In [39]:
credit_ordered.to_csv('cluster.csv')

In [40]:
pca = PCA(n_components=2)
principal_comp = pca.fit_transform(credicard_df_scaled)
principal_comp

In [41]:
pca_df = pd.DataFrame(data = principal_comp, columns=['pc1', 'pc2'])
pca_df.head()

In [42]:
pca_df = pd.concat([pca_df,  pd.DataFrame({'cluster':labels})], axis=1)
pca_df.head()

In [43]:
plt.figure(figsize=(10,10))
sns.scatterplot(x = 'pc1', y='pc2', hue='cluster',data=pca_df, palette=['red','green','blue','pink','yellow','gray','purple','black'])

In [44]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [45]:
input_df = Input(shape=(17,))
x = Dense(500, activation='relu')(input_df)
x = Dense(2000, activation='relu')(x)

encoded = Dense(10, activation='relu')(x)

x = Dense(2000, activation='relu')(encoded)
x = Dense(500, activation='relu')(x)

decoded = Dense(17)(x)


In [46]:
autoencoder = Model(input_df, decoded)

In [47]:
encoder = Model(input_df, encoded)

In [48]:
autoencoder.compile(optimizer = 'Adam', loss = 'mean_squared_error')

In [49]:
autoencoder.fit(credicard_df_scaled,credicard_df_scaled, epochs=50)

In [50]:
credicard_df_scaled.shape

In [51]:
compact = encoder.predict(credicard_df_scaled)

In [52]:
compact.shape

In [53]:
wcss_2 = []
range_values = range(1, 20)
for i in range_values:
  kmeans = KMeans(n_clusters=i)
  kmeans.fit(compact)
  wcss_2.append(kmeans.inertia_)

In [54]:
plt.plot(wcss_2, 'bx-')
plt.xlabel('Clusters')
plt.ylabel('WCSS')

In [55]:
plt.plot(wcss_1, 'bx-', color = 'r')
plt.plot(wcss_2, 'bx-', color = 'g');



In [56]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(compact)

In [57]:
labels = kmeans.labels_
labels, labels.shape

In [58]:
df_cluster_at = pd.concat([credicard_df, pd.DataFrame({'cluster': labels})], axis = 1)
df_cluster_at.head()

In [59]:
pca = PCA(n_components =  2)
prin_comp = pca.fit_transform(compact)
pca_df = pd.DataFrame(data = prin_comp, columns = ['pc1','pca2'])
pca_df.head()

In [60]:
pca_df = pd.concat([pca_df, pd.DataFrame({'cluster':labels})], axis=1)
pca_df.head()

In [61]:
plt.figure(figsize=(10,10))
sns.scatterplot(x = 'pc1', y='pca2', hue='cluster',data=pca_df, palette=['red','green','blue','pink'])

In [62]:
df_cluster_ordered = df_cluster_at.sort_values(by = 'cluster')
df_cluster_ordered.head()